In [2]:
import pandas as pd
import numpy as np
import os
import shutil
import time
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
from datetime import datetime
import glob
from itertools import permutations

# ==========================================
# 1. CONFIGURATION & SCHEMA
# ==========================================

LOCATIONS_FILE = 'locations_dummy.csv'
ALLOCATIONS_FILE = 'allocations.csv'
PARTS_FILE = 'synthetic_parts_generated.csv'
OUTPUT_DIR = 'validation_results'
MAX_EXECUTION_TIME_SEC = 300

REQUIRED_SCHEMA = {
    'LOCATIONS': {
        'file': LOCATIONS_FILE,
        # Ensure your locations file actually has these columns
        'columns': ['loc_inst_code', 'width', 'depth', 'height', 'x', 'y', 'z']
    },
    'ALLOCATIONS': {
        'file': ALLOCATIONS_FILE,
        # UPDATED COLUMN LIST TO MATCH YOUR NEW FORMAT
        'columns': ['loc_inst_code', 'LOCATION_TYPE', 'ITEM_ID', 
                    'QTY_ALLOCATED', 'MAX_UNITS', 
                    'GRID_X', 'GRID_Y', 'GRID_Z', 
                    'FULL_LAYERS', 'PARTIAL_UNITS',
                    'ORIENT_X_MM', 'ORIENT_Y_MM', 'ORIENT_Z_MM', 
                    'LOCATION_VOL_MM3', 'LOCATION_VOL_M3', 
                    'STORED_VOL_M3', 'UTILIZATION_PCT'] 
    },
    'PARTS': {
        'file': PARTS_FILE,
        'columns': ['ITEM_ID', 'LEN_MM', 'WID_MM', 'DEP_MM', 'WT_KG', 
                    'BOXES_ON_HAND', 'QTY_PER_BOX']
    }
}

# ==========================================
# 2. LOGGING & UTILS
# ==========================================

class Colors:
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    CYAN = '\033[96m'
    RESET = '\033[0m'
    BOLD = '\033[1m'

report_buffer = []

def log(message):
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    report_buffer.append(f"[{timestamp}] {message}")
    
    c_msg = message
    if "PASS" in message: c_msg = message.replace("PASS", f"{Colors.GREEN}{Colors.BOLD}PASS{Colors.RESET}")
    elif "FAIL" in message: c_msg = message.replace("FAIL", f"{Colors.RED}{Colors.BOLD}FAIL{Colors.RESET}")
    elif "CRITICAL" in message: c_msg = f"{Colors.RED}{Colors.BOLD}{message}{Colors.RESET}"
    elif "WARN" in message: c_msg = message.replace("WARN", f"{Colors.YELLOW}{Colors.BOLD}WARN{Colors.RESET}")
    elif "---" in message: c_msg = f"{Colors.CYAN}{message}{Colors.RESET}"
    
    print(f"[{timestamp}] {c_msg}")

def setup_environment():
    if os.path.exists(OUTPUT_DIR):
        shutil.rmtree(OUTPUT_DIR)
    os.makedirs(OUTPUT_DIR)
    log(f"Output folder '{OUTPUT_DIR}' ready.")

# ==========================================
# 3. DATA LOADING
# ==========================================

def load_and_validate_dataset(key, config):
    filepath = config['file']
    log(f"--- Loading {key} ({filepath}) ---")
    
    if not os.path.exists(filepath):
        log(f"CRITICAL: File {filepath} not found.")
        return None, False

    try:
        df = pd.read_csv(filepath, sep=None, engine='python', dtype=str)
        df.columns = df.columns.str.strip().str.replace('^ï»¿', '', regex=True)
        
        # Add Original Row ID
        df['ROW_ID'] = df.index + 2 

        missing = [c for c in config['columns'] if c not in df.columns]
        if missing:
            log(f"CRITICAL SCHEMA ERROR in {key}. Missing: {missing}")
            return df, False
        
        log(f"SUCCESS: {key} loaded ({len(df)} rows).")
        return df, True
    except Exception as e:
        log(f"CRITICAL ERROR reading {filepath}: {e}")
        return None, False

def convert_numeric(df, cols):
    for col in cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    return df

def load_all_data():
    setup_environment()
    datasets = {}
    valid_flags = {}
    
    # Locations
    df, valid = load_and_validate_dataset('LOCATIONS', REQUIRED_SCHEMA['LOCATIONS'])
    if valid: df = convert_numeric(df, ['width', 'depth', 'height', 'x', 'y', 'z'])
    datasets['LOCATIONS'] = df
    valid_flags['LOCATIONS'] = valid

    # Allocations
    df, valid = load_and_validate_dataset('ALLOCATIONS', REQUIRED_SCHEMA['ALLOCATIONS'])
    if valid: 
        # UPDATED NUMERIC COLUMNS
        df = convert_numeric(df, ['GRID_X', 'GRID_Y', 'GRID_Z', 
                                  'ORIENT_X_MM', 'ORIENT_Y_MM', 'ORIENT_Z_MM', 
                                  'MAX_UNITS', 'QTY_ALLOCATED', 
                                  'LOCATION_VOL_MM3', 'UTILIZATION_PCT'])
    datasets['ALLOCATIONS'] = df
    valid_flags['ALLOCATIONS'] = valid

    # Parts
    df, valid = load_and_validate_dataset('PARTS', REQUIRED_SCHEMA['PARTS'])
    if valid: df = convert_numeric(df, ['LEN_MM', 'WID_MM', 'DEP_MM', 'WT_KG', 
                                        'BOXES_ON_HAND', 'QTY_PER_BOX'])
    datasets['PARTS'] = df
    valid_flags['PARTS'] = valid
    
    return datasets, valid_flags

# ==========================================
# 4. VALIDATION LOGIC
# ==========================================

def estimate_and_sample(df, check_name, validation_func, *args):
    log(f"Starting {check_name}...")
    total = len(df)
    test_size = min(1000, total)
    if test_size == 0: return pd.DataFrame()

    t0 = time.time()
    try:
        _ = validation_func(df.head(test_size), *args, quiet=True)
    except Exception as e:
        log(f"ERROR in {check_name}: {e}")
        # raise e # Uncomment for debugging
        return pd.DataFrame()
        
    dur = time.time() - t0
    if dur == 0: dur = 0.001
    est_time = (dur / test_size) * total
    log(f"   Est. time: {est_time:.2f}s")

    if est_time > MAX_EXECUTION_TIME_SEC:
        safe_rows = int((MAX_EXECUTION_TIME_SEC / dur) * test_size)
        log(f"   WARN: Time limit exceeded. Sampling {safe_rows} rows.")
        df_to_process = df.sample(n=safe_rows, random_state=42)
    else:
        df_to_process = df

    return validation_func(df_to_process, *args, quiet=False)

# --- CHECK: Stated Utilization Accuracy ---
# --- CHECK: Stated Utilization Accuracy ---
def check_stated_utilization(df_alloc, df_loc, quiet=False):
    col_name = 'UTILIZATION_PCT'
    
    if col_name not in df_alloc.columns:
        if not quiet: log(f"WARN: Column '{col_name}' not found.")
        return pd.DataFrame()

    if not quiet: log(f"INFO: Checking utilization accuracy using column: '{col_name}'")

    merged = df_alloc.merge(df_loc, on='loc_inst_code', how='left', suffixes=('', '_LOC'))
    
    issues = []
    iterator = tqdm(merged.iterrows(), total=merged.shape[0]) if not quiet else merged.iterrows()
    
    for idx, row in iterator:
        if pd.isna(row['width']): continue

        # --- FIX STARTS HERE ---
        
        # 1. Calculate volume of ONE unit based on its orientation
        one_unit_vol = row['ORIENT_X_MM'] * row['ORIENT_Y_MM'] * row['ORIENT_Z_MM']
        
        # 2. Multiply by the ACTUAL quantity in the bin (not the Grid capacity)
        actual_occupied_vol = one_unit_vol * row['QTY_ALLOCATED']
        
        # --- FIX ENDS HERE ---
        
        loc_vol = row['width'] * row['depth'] * row['height']
        
        if loc_vol <= 0: continue

        # Calculate percentage based on actual inventory
        calc_pct = (actual_occupied_vol / loc_vol) * 100
        stated_pct = row[col_name]

        # Tolerance: 1% difference
        if abs(calc_pct - stated_pct) > 1.0:
            issues.append({
                'ROW_ID': row['ROW_ID'],
                'loc_inst_code': row['loc_inst_code'],
                'Issue': 'Utilization Data Mismatch',
                # Updated error message to be clearer
                'Details': f"Stated: {stated_pct:.1f}% != Calc: {calc_pct:.1f}% (Qty {row['QTY_ALLOCATED']} * Vol / Bin Vol)"
            })
            
    return pd.DataFrame(issues)

# --- CHECK: Stack vs Bin Dimensions (Geometric Fit) ---
def check_stack_fit(df_alloc, df_loc, quiet=False):
    # UPDATED: Merge on 'loc_inst_code'
    merged = df_alloc.merge(df_loc, on='loc_inst_code', how='left', suffixes=('', '_LOC'))
    issues = []
    
    iterator = tqdm(merged.iterrows(), total=merged.shape[0]) if not quiet else merged.iterrows()
    
    for idx, row in iterator:
        if pd.isna(row['width']): continue 

        stack_x = row['GRID_X'] * row['ORIENT_X_MM']
        stack_y = row['GRID_Y'] * row['ORIENT_Y_MM']
        stack_z = row['GRID_Z'] * row['ORIENT_Z_MM']
        
        tol = 1.0
        if (stack_x > row['width'] + tol) or (stack_y > row['depth'] + tol) or (stack_z > row['height'] + tol):
            issues.append({
                'ROW_ID': row['ROW_ID'],
                'loc_inst_code': row['loc_inst_code'],
                'ITEM_ID': row['ITEM_ID'], # Updated from SKU
                'Issue': 'Stack Exceeds Bin',
                'Details': f"Bin: {row['width']}x{row['depth']}x{row['height']} | Stack: {stack_x:.1f}x{stack_y:.1f}x{stack_z:.1f}"
            })
    return pd.DataFrame(issues)

# --- CHECK: Inventory Balance (Partial Unallocated) ---
def check_inventory_balance(df_alloc, df_parts):
    log("--- Check: Inventory Balance (Partial Unallocated) ---")
    
    # FIX: Removed * df_parts['QTY_PER_BOX'] to ensure we count boxes, not contents
    df_parts['TOTAL_DEMAND'] = df_parts['BOXES_ON_HAND'] 
    
    demand_map = df_parts.set_index('ITEM_ID')['TOTAL_DEMAND'].to_dict()
    
    # Ensure numeric
    df_alloc['QTY_ALLOCATED'] = pd.to_numeric(df_alloc['QTY_ALLOCATED'], errors='coerce').fillna(0)
    alloc_sums = df_alloc.groupby('ITEM_ID')['QTY_ALLOCATED'].sum().to_dict()
    
    issues = []
    for item_id, demand in demand_map.items():
        allocated = alloc_sums.get(item_id, 0)
        
        # Use simple float tolerance just in case
        if allocated < (demand - 0.01):
            issues.append({'ITEM_ID': item_id, 'Issue': 'Partial/No Allocation', 'Demand': demand, 'Allocated': allocated, 'Missing': demand - allocated})
        elif allocated > (demand + 0.01):
            issues.append({'ITEM_ID': item_id, 'Issue': 'Over-Allocation', 'Demand': demand, 'Allocated': allocated, 'Excess': allocated - demand})
            
    df_issues = pd.DataFrame(issues)
    if not df_issues.empty:
        log(f"FAIL: Found {len(df_issues)} Items with inventory mismatches.")
        df_issues.to_csv(f"{OUTPUT_DIR}/fail_inventory_balance.csv", index=False)
    else:
        log("PASS: Total allocated boxes match inventory on hand.")

# --- CHECK: Volume Data Integrity ---
def check_volume_data_integrity(df_alloc, df_loc, quiet=False):
    # UPDATED: Merge on loc_inst_code
    merged = df_alloc.merge(df_loc, on='loc_inst_code', how='left', suffixes=('', '_LOC'))
    issues = []
    iterator = tqdm(merged.iterrows(), total=merged.shape[0]) if not quiet else merged.iterrows()
    for idx, row in iterator:
        if pd.isna(row['width']): continue
        real_vol = row['width'] * row['depth'] * row['height']
        
        # Using LOCATION_VOL_MM3 from new format
        if abs(real_vol - row['LOCATION_VOL_MM3']) > 1.0:
            issues.append({
                'ROW_ID': row['ROW_ID'],
                'loc_inst_code': row['loc_inst_code'],
                'Issue': 'Volume Data Error',
                'Details': f"Stated: {row['LOCATION_VOL_MM3']} != Real: {real_vol}"
            })
    return pd.DataFrame(issues)

# --- CHECK: Grid Math ---
def check_grid_consistency(df_alloc, quiet=False):
    issues = []
    iterator = tqdm(df_alloc.iterrows(), total=df_alloc.shape[0]) if not quiet else df_alloc.iterrows()
    for idx, row in iterator:
        grid_cap = row['GRID_X'] * row['GRID_Y'] * row['GRID_Z']
        if abs(grid_cap - row['MAX_UNITS']) > 0.1:
            issues.append({
                'ROW_ID': row['ROW_ID'], 
                'loc_inst_code': row['loc_inst_code'], 
                'Issue': 'Grid Math Mismatch', 
                'Details': f"Grid ({grid_cap}) != MaxUnits ({row['MAX_UNITS']})"
            })
    return pd.DataFrame(issues)

# --- CHECK: Rigid Body ---
def check_rigid_body(df_alloc, df_parts, quiet=False):
    # UPDATED: Merge on ITEM_ID
    merged = df_alloc.merge(df_parts, on='ITEM_ID', how='left', suffixes=('', '_PART'))
    issues = []
    iterator = tqdm(merged.iterrows(), total=merged.shape[0]) if not quiet else merged.iterrows()
    for idx, row in iterator:
        if pd.isna(row['LEN_MM']): continue
        od = sorted([row['ORIENT_X_MM'], row['ORIENT_Y_MM'], row['ORIENT_Z_MM']])
        pd_ = sorted([row['LEN_MM'], row['WID_MM'], row['DEP_MM']])
        if any(abs(o - p) > 0.5 for o, p in zip(od, pd_)):
            issues.append({
                'ROW_ID': row['ROW_ID'], 
                'loc_inst_code': row['loc_inst_code'], 
                'Issue': 'Dimensions Morphing', 
                'Details': f"Alloc {od} vs Part {pd_}"
            })
    return pd.DataFrame(issues)

# --- CHECK: Bin Overlap ---
def check_bin_overlaps(df_loc, quiet=False):
    df = df_loc.copy()
    df['x2'] = df['x'] + df['width']
    df['y2'] = df['y'] + df['depth']
    df['z2'] = df['z'] + df['height']
    df.sort_values('x', inplace=True)
    issues = []
    data = df.to_dict('records')
    n = len(data)
    if n == 0: return pd.DataFrame()
    iter_range = tqdm(range(n), total=n) if not quiet else range(n)
    for i in iter_range:
        a = data[i]
        for j in range(i + 1, n):
            b = data[j]
            if b['x'] >= a['x2'] - 0.1: break
            if (a['y'] < b['y2']) and (a['y2'] > b['y']) and (a['z'] < b['z2']) and (a['z2'] > b['z']):
                issues.append({'LOC_A': a['loc_inst_code'], 'LOC_B': b['loc_inst_code'], 'Issue': 'Physical Overlap', 'Details': f"A({a['x']}) vs B({b['x']})"})
    return pd.DataFrame(issues)


def calculate_max_storage_capacity(item_dims, bin_dims):
    """
    Returns max quantity of item_dims (L,W,D) that fit in bin_dims (W,D,H)
    by testing all 6 rotation permutations.
    """
    L, W, D = item_dims
    bin_W, bin_D, bin_H = bin_dims
    
    # 1. Get all 6 unique rotations (e.g., LWD, LDW, WLD, etc.)
    unique_orientations = list(set(permutations([L, W, D])))
    
    max_qty = 0
    best_fail_reason = "Dimensions too large"

    for orient in unique_orientations:
        ox, oy, oz = orient
        
        # Check geometric fit (does 1 unit fit?)
        if ox <= bin_W and oy <= bin_D and oz <= bin_H:
            # Calculate bulk fit (how many units fit?)
            nx = int(bin_W // ox)
            ny = int(bin_D // oy)
            nz = int(bin_H // oz)
            total = nx * ny * nz
            
            if total > max_qty:
                max_qty = total

    # Logic to return a specific reason if it fits nowhere
    if max_qty == 0:
        # Compare sorted dimensions to give the most "charitable" fail reason
        s_item = sorted([L, W, D])
        s_bin = sorted([bin_W, bin_D, bin_H])
        if s_item[0] > s_bin[0]: best_fail_reason = f"Min Dim {s_item[0]} > Bin Min {s_bin[0]}"
        elif s_item[1] > s_bin[1]: best_fail_reason = f"Mid Dim {s_item[1]} > Bin Mid {s_bin[1]}"
        elif s_item[2] > s_bin[2]: best_fail_reason = f"Max Dim {s_item[2]} > Bin Max {s_bin[2]}"

    return max_qty, best_fail_reason

# --- CHECK: Unallocated Items ---
def check_unallocated_feasibility(df_parts, df_alloc, df_loc):
    log("--- Check: Unallocated Feasibility (6-Axis Geometric & Bulk) ---")
    
    # 1. Identify Unallocated Items
    all_items = set(df_parts['ITEM_ID'])
    alloc_items = set(df_alloc['ITEM_ID'])
    unallocated_ids = list(all_items - alloc_items)
    
    if not unallocated_ids:
        log("PASS: All items are allocated.")
        return

    # 2. Identify EMPTY Bins
    occupied_locs = set(df_alloc['loc_inst_code'])
    empty_locs = df_loc[~df_loc['loc_inst_code'].isin(occupied_locs)].copy()
    
    if empty_locs.empty:
        log(f"WARN: {len(unallocated_ids)} unallocated items, but NO empty bins exist.")
        return

    # 3. OPTIMIZATION: Group empty bins by unique dimensions
    # We create a signature string "WxDxH" to find unique bin types
    empty_locs['dim_sig'] = empty_locs.apply(lambda x: f"{x['width']}_{x['depth']}_{x['height']}", axis=1)
    unique_bins = empty_locs.drop_duplicates(subset='dim_sig')
    
    log(f"INFO: Checking {len(unallocated_ids)} items against {len(unique_bins)} unique empty bin types.")
    
    feasible_unallocated = []
    true_failures = []

    for item_id in unallocated_ids:
        # Get Item Dimensions
        part = df_parts[df_parts['ITEM_ID'] == item_id].iloc[0]
        item_dims = (part['LEN_MM'], part['WID_MM'], part['DEP_MM'])
        
        found_fit = False
        last_reason = "Unknown"
        
        # Check against every UNIQUE empty bin type
        for _, bin_row in unique_bins.iterrows():
            bin_dims = (bin_row['width'], bin_row['depth'], bin_row['height'])
            
            # Run the helper function
            max_qty, reason = calculate_max_storage_capacity(item_dims, bin_dims)
            
            if max_qty > 0:
                # IT FITS!
                feasible_unallocated.append({
                    'ITEM_ID': item_id,
                    'Item_Dims': item_dims,
                    'Fits_In_Bin_Type': bin_row['dim_sig'],
                    'Max_Possible_Qty': max_qty
                })
                found_fit = True
                break # Stop checking other bins for this item, we proved it could fit.
            else:
                last_reason = reason

        if not found_fit:
            true_failures.append({
                'ITEM_ID': item_id,
                'Item_Dims': item_dims,
                'Reason': last_reason
            })

    # 4. Reporting
    if len(feasible_unallocated) > 0:
        log(f"FAIL: {len(feasible_unallocated)} Unallocated Items COULD fit in currently empty bins.")
        df_fail = pd.DataFrame(feasible_unallocated)
        print(df_fail.head().to_string(index=False))
        df_fail.to_csv(f"{OUTPUT_DIR}/fail_unallocated_feasible.csv", index=False)
    else:
        log(f"PASS: Correct. All {len(true_failures)} unallocated items physically cannot fit in any available empty bin.")
        if len(true_failures) > 0:
            df_pass = pd.DataFrame(true_failures)
            df_pass.to_csv(f"{OUTPUT_DIR}/pass_unallocated_impossible.csv", index=False)

# ==========================================
# 5. VISUALIZATION
# ==========================================

def plot_single_bin(row, title_prefix, filename_tag):
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(2, 2, width_ratios=[2, 1])
    ax1 = fig.add_subplot(gs[0, 0]); ax2 = fig.add_subplot(gs[1, 0]); ax_text = fig.add_subplot(gs[:, 1]); ax_text.axis('off')
    
    bin_w, bin_h, bin_d = row['width'], row['height'], row['depth']
    item_w, item_h, item_d = row['ORIENT_X_MM'], row['ORIENT_Z_MM'], row['ORIENT_Y_MM']
    gx, gy, gz = int(row['GRID_X']), int(row['GRID_Y']), int(row['GRID_Z'])
    
    # --- LOGIC FIX: Determine which slots are ACTUALLY occupied ---
    qty_alloc = int(row['QTY_ALLOCATED'])
    occupied_slots = []
    count = 0
    
    # Simulate filling order: X (Width) -> Y (Depth) -> Z (Height)
    for z in range(gz):
        for y in range(gy):
            for x in range(gx):
                if count < qty_alloc:
                    occupied_slots.append((x, y, z))
                    count += 1
                else:
                    break
    
    # Create sets of occupied 2D coordinates for the projections
    # Front View (X, Z) - Draw if an item exists at this X,Z (at any depth)
    valid_front_xz = set((x, z) for x, y, z in occupied_slots)
    # Top View (X, Y) - Draw if an item exists at this X,Y (at any height)
    valid_top_xy = set((x, y) for x, y, z in occupied_slots)

    # --- DRAW FRONT VIEW (X-Z) ---
    ax1.add_patch(patches.Rectangle((0, 0), bin_w, bin_h, fill=False, edgecolor='red', lw=3, label='Bin'))
    for z in range(gz):
        for x in range(gx):
            # Only draw if this slot actually contains an item
            if (x, z) in valid_front_xz:
                ax1.add_patch(patches.Rectangle((x*item_w, z*item_h), item_w, item_h, lw=1, ec='black', fc='skyblue', alpha=0.6))
    
    ax1.set_title("FRONT VIEW (X-Z) - ACTUAL INVENTORY", fontsize=12, fontweight='bold')
    ax1.set_xlim(-50, bin_w + 50); ax1.set_ylim(-50, bin_h + 50); ax1.set_aspect('equal'); ax1.grid(True, linestyle=':', alpha=0.5)

    # --- DRAW TOP VIEW (X-Y) ---
    ax2.add_patch(patches.Rectangle((0, 0), bin_w, bin_d, fill=False, edgecolor='red', lw=3))
    for y in range(gy):
        for x in range(gx):
            # Only draw if this slot actually contains an item
            if (x, y) in valid_top_xy:
                ax2.add_patch(patches.Rectangle((x*item_w, y*item_d), item_w, item_d, lw=1, ec='black', fc='orange', alpha=0.6))
    
    ax2.set_title("TOP VIEW (X-Y) - ACTUAL INVENTORY", fontsize=12, fontweight='bold')
    ax2.set_xlim(-50, bin_w + 50); ax2.set_ylim(-50, bin_d + 50); ax2.set_aspect('equal'); ax2.grid(True, linestyle=':', alpha=0.5)

    info_text = (
        f"REPORT: {title_prefix}\n----------------------------------------\n"
        f"ALLOCATION ROW:   {row['ROW_ID']}\nLOCATION ID:      {row['loc_inst_code']}\nITEM ID:          {row['ITEM_ID']}\n"
        f"----------------------------------------\nMETRICS:\n"
        f"  Utilization:    {row['UTILIZATION_PCT']:.2f} % (Based on Qty {qty_alloc})\n"
        f"  Qty Allocated:  {qty_alloc}\n"
        f"  Max Capacity:   {row['MAX_UNITS']}\n\n"
        f"DIMENSIONS (mm):\n  Bin (WxDxH):    {bin_w} x {bin_d} x {bin_h}\n"
        f"  Item Orient:    {row['ORIENT_X_MM']} x {row['ORIENT_Y_MM']} x {row['ORIENT_Z_MM']}\n\n"
        f"VOLUMES (mm3):\n  Bin Volume:     {row['LOC_VOL']:,.0f}\n  Allocated Vol:  {row['STACK_VOL']:,.0f}\n"
    )
    ax_text.text(0.05, 0.95, info_text, transform=ax_text.transAxes, fontsize=12, verticalalignment='top', family='monospace', bbox=dict(boxstyle="round,pad=0.5", fc="white", ec="gray", alpha=0.9))
    plt.tight_layout(); save_path = f"{OUTPUT_DIR}/{filename_tag}.png"; plt.savefig(save_path); plt.close(); log(f"Plot saved to {save_path}")

def visualize_utilization_extremes(datasets):
    log("--- Generating Utilization Plots ---")
    df_alloc = datasets['ALLOCATIONS']; df_loc = datasets['LOCATIONS']
    
    # Merge on loc_inst_code
    merged = df_alloc.merge(df_loc, on='loc_inst_code', how='inner', suffixes=('', '_LOC'))
    if merged.empty: return
    
    # RECALCULATE VOLUMES based on ACTUAL QTY (Not Grid Capacity)
    merged['STACK_VOL'] = merged['QTY_ALLOCATED'] * (merged['ORIENT_X_MM'] * merged['ORIENT_Y_MM'] * merged['ORIENT_Z_MM'])
    merged['LOC_VOL'] = merged['width'] * merged['depth'] * merged['height']
    
    # Filter valid bins
    merged = merged[merged['LOC_VOL'] > 0]
    
    # FORCE CALCULATION of Real Utilization %
    # This ensures we rank by what is actually in the box, matching the new plot logic
    merged['UTILIZATION_PCT'] = (merged['STACK_VOL'] / merged['LOC_VOL']) * 100
    
    # Plot Max
    top_row = merged.sort_values(by='UTILIZATION_PCT', ascending=False).iloc[0]
    plot_single_bin(top_row, "HIGHEST UTILIZATION", "visual_utilization_max")
    
    # Plot Min (active only)
    active = merged[merged['UTILIZATION_PCT'] > 0]
    if not active.empty:
        low_row = active.sort_values(by='UTILIZATION_PCT', ascending=True).iloc[0]
        plot_single_bin(low_row, "LOWEST UTILIZATION", "visual_utilization_min")

# ==========================================
# 6. RUNNER
# ==========================================

def run_full_diagnostic():
    datasets, valid_flags = load_all_data()
    if not all(valid_flags.values()): log("STOPPING: Schema Errors."); return

    # 1. Volume Data Integrity
    log("--- Check: Volume Data Integrity ---")
    res = estimate_and_sample(datasets['ALLOCATIONS'], "Volume Data", check_volume_data_integrity, datasets['LOCATIONS'])
    if not res.empty: log(f"FAIL: {len(res)} Volume Data mismatches."); res.to_csv(f"{OUTPUT_DIR}/fail_volume_data.csv", index=False)
    else: log("PASS: Stated volumes match location dimensions.")
    
    # 2. Utilization Data Integrity
    log("--- Check: Stated Utilization Accuracy ---")
    res = estimate_and_sample(datasets['ALLOCATIONS'], "Utilization Data", check_stated_utilization, datasets['LOCATIONS'])
    if not res.empty: log(f"FAIL: {len(res)} Utilization % mismatches."); res.to_csv(f"{OUTPUT_DIR}/fail_utilization_data.csv", index=False)
    else: log("PASS: Utilization data matches calculations.")

    # 3. Inventory Balance
    check_inventory_balance(datasets['ALLOCATIONS'], datasets['PARTS'])

    # 4. Standard Checks
    log("--- Check: Single SKU per Bin ---")
    # UPDATED: using loc_inst_code
    dupes = datasets['ALLOCATIONS'][datasets['ALLOCATIONS'].duplicated(subset=['loc_inst_code'], keep=False)]
    if not dupes.empty: log(f"FAIL: {len(dupes)} locations have multiple SKUs."); dupes.to_csv(f"{OUTPUT_DIR}/fail_single_sku.csv")
    else: log("PASS: Single SKU constraint met.")

    log("--- Check: Grid Math ---")
    res = estimate_and_sample(datasets['ALLOCATIONS'], "Grid Math", check_grid_consistency)
    if not res.empty: log(f"FAIL: {len(res)} Grid Math errors."); res.to_csv(f"{OUTPUT_DIR}/fail_grid_math.csv", index=False)
    else: log("PASS: Grid Math consistent.")

    log("--- Check: Rigid Body ---")
    res = estimate_and_sample(datasets['ALLOCATIONS'], "Rigid Body", check_rigid_body, datasets['PARTS'])
    if not res.empty: log(f"FAIL: {len(res)} Rigid Body errors."); res.to_csv(f"{OUTPUT_DIR}/fail_rigid_body.csv", index=False)
    else: log("PASS: Rigid Body dimensions valid.")

    log("--- Check: Stack vs Bin Dimensions ---")
    res = estimate_and_sample(datasets['ALLOCATIONS'], "Stack Fit", check_stack_fit, datasets['LOCATIONS'])
    if not res.empty: log(f"FAIL: {len(res)} allocations exceed bin dimensions."); res.to_csv(f"{OUTPUT_DIR}/fail_stack_fit.csv", index=False)
    else: log("PASS: All stacks fit within bins.")

    log("--- Check: Bin Overlaps ---")
    res = estimate_and_sample(datasets['LOCATIONS'], "Bin Overlap", check_bin_overlaps)
    if not res.empty: log(f"FAIL: {len(res)} locations overlap physically."); res.to_csv(f"{OUTPUT_DIR}/fail_bin_overlap.csv", index=False)
    else: log("PASS: No bin overlaps.")

    check_unallocated_feasibility(datasets['PARTS'], datasets['ALLOCATIONS'], datasets['LOCATIONS'])
    visualize_utilization_extremes(datasets)

    with open(f"{OUTPUT_DIR}/validation_report.txt", "w") as f: f.write("\n".join(report_buffer))
    log("Validation Complete.")

if __name__ == "__main__":
    run_full_diagnostic()

[2026-01-06 16:17:20] Output folder 'validation_results' ready.
[2026-01-06 16:17:20] --- Loading LOCATIONS (locations_dummy.csv) ---
[2026-01-06 16:17:20] SUCCESS: LOCATIONS loaded (357 rows).
[2026-01-06 16:17:20] --- Loading ALLOCATIONS (allocations.csv) ---
[2026-01-06 16:17:20] SUCCESS: ALLOCATIONS loaded (174 rows).
[2026-01-06 16:17:20] --- Loading PARTS (synthetic_parts_generated.csv) ---
[2026-01-06 16:17:20] SUCCESS: PARTS loaded (67 rows).
[2026-01-06 16:17:20] --- Check: Volume Data Integrity ---
[2026-01-06 16:17:20] Starting Volume Data...
[2026-01-06 16:17:20]    Est. time: 0.02s


100%|██████████████████████████████████████████████████████████████████████████████| 174/174 [00:00<00:00, 6815.55it/s]


[2026-01-06 16:17:20] PASS: Stated volumes match location dimensions.
[2026-01-06 16:17:20] --- Check: Stated Utilization Accuracy ---
[2026-01-06 16:17:20] Starting Utilization Data...
[2026-01-06 16:17:20]    Est. time: 0.03s
[2026-01-06 16:17:20] INFO: Checking utilization accuracy using column: 'UTILIZATION_PCT'


100%|██████████████████████████████████████████████████████████████████████████████| 174/174 [00:00<00:00, 8083.93it/s]


[2026-01-06 16:17:20] PASS: Utilization data matches calculations.
[2026-01-06 16:17:20] --- Check: Inventory Balance (Partial Unallocated) ---
[2026-01-06 16:17:20] PASS: Total allocated boxes match inventory on hand.
[2026-01-06 16:17:20] --- Check: Single SKU per Bin ---
[2026-01-06 16:17:20] PASS: Single SKU constraint met.
[2026-01-06 16:17:20] --- Check: Grid Math ---
[2026-01-06 16:17:20] Starting Grid Math...
[2026-01-06 16:17:20]    Est. time: 0.02s


...

[2026-01-06 16:17:20] PASS: Rigid Body dimensions valid.
[2026-01-06 16:17:20] --- Check: Stack vs Bin Dimensions ---
[2026-01-06 16:17:20] Starting Stack Fit...
[2026-01-06 16:17:20]    Est. time: 0.03s


100%|██████████████████████████████████████████████████████████████████████████████| 174/174 [00:00<00:00, 7555.66it/s]


[2026-01-06 16:17:20] PASS: All stacks fit within bins.
[2026-01-06 16:17:20] --- Check: Bin Overlaps ---
[2026-01-06 16:17:20] Starting Bin Overlap...
[2026-01-06 16:17:20]    Est. time: 0.02s


100%|█████████████████████████████████████████████████████████████████████████████| 357/357 [00:00<00:00, 53544.31it/s]


[2026-01-06 16:17:20] PASS: No bin overlaps.
[2026-01-06 16:17:20] --- Check: Unallocated Feasibility (6-Axis Geometric & Bulk) ---
[2026-01-06 16:17:20] PASS: All items are allocated.
[2026-01-06 16:17:20] --- Generating Utilization Plots ---
[2026-01-06 16:17:21] Plot saved to validation_results/visual_utilization_max.png
[2026-01-06 16:17:22] Plot saved to validation_results/visual_utilization_min.png
[2026-01-06 16:17:22] Validation Complete.
